In [1]:
import pandas as pd

# = = = = = = = = = = = = = = =

from urllib.parse import urlparse, parse_qs, urlencode

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

In [2]:
list_series_error, df_input = [], pd.read_excel('./file/input.xlsx',
                                                header=0,
                                                dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：2



,No,Url
0,1,https://www.cardone.com/motors/wiper-and-washe...
1,2,https://www.cardone.com/drivetrain/driveline-a...


In [3]:
(crawler_index, crawler_series) = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                     1
 Url    https://www.cardone.com/motors/wiper-and-washe...
 Name: 0, dtype: object)

In [4]:
import time

url_parse = urlparse(crawler_series['Url'])

dict_param = parse_qs(url_parse.query)
dict_param.pop('page', None)
dict_param['sort'] = 'alphaasc'
dict_param['limit'] = '96'

url_base = f'{url_parse.scheme}://{url_parse.netloc}{url_parse.path}?{urlencode(dict_param, doseq=True)}'

list_url, page = [], 1
while True:

    count_retry = 0
    while True:
        count_retry += 1
        print(page, count_retry)
        try:
            url_request = f'{url_base}&page={page}'

            resp = requests.get(url_request,
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(5, 15))

            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                list_href = html.xpath('//h4[@class="card-title NoMargin"]/a/@href')
                if list_href:
                    if page == 1:
                        with open('./url.html', 'w', encoding='UTF-8') as file:
                            file.write(str(soup))
                    break
        except KeyboardInterrupt:
            break
        except:
            time.sleep(3)
            continue

    # = = = = = = = = = = = = = = =

    list_url.extend(list_href)
    list_url = list(dict.fromkeys(list_url))

    # = = = = = = = = = = = = = = =

    page += 1

    # = = = = = = = = = = = = = = =

    if not html.xpath('//li[@class="pagination-item pagination-item--next"]'):
        break

list_url

1 1
2 1
3 1
3 2
4 1
5 1
5 2
5 3
6 1
7 1
8 1
9 1
9 2
10 1
11 1
12 1
13 1
13 2
14 1
15 1


['https://www.cardone.com/product/windshield-wiper-motor-40-1000',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10000',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10005',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10006',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10007',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10009',
 'https://www.cardone.com/product/windshield-wiper-motor-40-1001',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10013',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10014',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10015',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10016',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10019',
 'https://www.cardone.com/product/windshield-wiper-motor-40-1002',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10020',
 'https://www.cardone.com/product/windshield-wiper-

In [5]:
list_url

['https://www.cardone.com/product/windshield-wiper-motor-40-1000',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10000',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10005',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10006',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10007',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10009',
 'https://www.cardone.com/product/windshield-wiper-motor-40-1001',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10013',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10014',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10015',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10016',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10019',
 'https://www.cardone.com/product/windshield-wiper-motor-40-1002',
 'https://www.cardone.com/product/windshield-wiper-motor-40-10020',
 'https://www.cardone.com/product/windshield-wiper-

In [6]:
df_correct = pd.DataFrame({'No': range(1, len(list_url) + 1),
                           'Url': list_url})
df_correct.to_excel('./test_url.xlsx', index=False)

df_correct

,No,Url
0,1,https://www.cardone.com/product/windshield-wip...
1,2,https://www.cardone.com/product/windshield-wip...
2,3,https://www.cardone.com/product/windshield-wip...
3,4,https://www.cardone.com/product/windshield-wip...
4,5,https://www.cardone.com/product/windshield-wip...
...,...,...
1396,1397,https://www.cardone.com/product/windshield-wip...
1397,1398,https://www.cardone.com/product/windshield-wip...
1398,1399,https://www.cardone.com/product/windshield-wip...
1399,1400,https://www.cardone.com/product/windshield-wip...


In [7]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://www.cardone.com/motors/wiper-and-washe...
 Request_Url    https://www.cardone.com/motors/wiper-and-washe...
 Name: 0, dtype: object]

In [8]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_url_error.xlsx', index=False)

df_error

,No,Url,Request_Url
0,1,https://www.cardone.com/motors/wiper-and-washe...,https://www.cardone.com/motors/wiper-and-washe...
